In [1]:
!nvidia-smi

Tue Sep 13 13:52:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!unzip -u "/content/drive/MyDrive/OCT Project/OCT Code & Dataset/Image Processing/Alpha Beta on Median Erosion/DRUSENA.zip" -d "/content/OCT_Data"

print('\n\ndone!')

Streaming output truncated to the last 5000 lines.
  inflating: /content/OCT_Data/DRUSENA/DRU_Al 5500.jpeg  
  inflating: /content/OCT_Data/DRUSENA/DRU_Al 5501.jpeg  
  inflating: /content/OCT_Data/DRUSENA/DRU_Al 5502.jpeg  
  inflating: /content/OCT_Data/DRUSENA/DRU_Al 5503.jpeg  
  inflating: /content/OCT_Data/DRUSENA/DRU_Al 5504.jpeg  
  inflating: /content/OCT_Data/DRUSENA/DRU_Al 5505.jpeg  
  inflating: /content/OCT_Data/DRUSENA/DRU_Al 5506.jpeg  
  inflating: /content/OCT_Data/DRUSENA/DRU_Al 5507.jpeg  
  inflating: /content/OCT_Data/DRUSENA/DRU_Al 5508.jpeg  
  inflating: /content/OCT_Data/DRUSENA/DRU_Al 5509.jpeg  
  inflating: /content/OCT_Data/DRUSENA/DRU_Al 551.jpeg  
  inflating: /content/OCT_Data/DRUSENA/DRU_Al 5510.jpeg  
  inflating: /content/OCT_Data/DRUSENA/DRU_Al 5511.jpeg  
  inflating: /content/OCT_Data/DRUSENA/DRU_Al 5512.jpeg  
  inflating: /content/OCT_Data/DRUSENA/DRU_Al 5513.jpeg  
  inflating: /content/OCT_Data/DRUSENA/DRU_Al 5514.jpeg  
  inflating: /content/

In [7]:
import cv2
import numpy as np
import glob
import os

In [8]:
pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import splitfolders
import os

input_folder="/content/OCT_Data"


output="/content/OCT_AB"

splitfolders.ratio(input_folder, output, seed=42, ratio=(.75,.1,.15)) ### train 75%, val 10%, test 15%

Copying files: 131148 files [00:25, 5150.55 files/s]


In [10]:
####### transfer learning model

################# import various transfer elarning models ###################

#from tensorflow.keras.applications.inception_v3 import InceptionV3
#from tensorflow.keras.applications.vgg16 import VGG16
#from tensorflow.keras.applications.vgg19 import VGG19
#from tensorflow.keras.applications import Xception


#############################################################################

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.applications import ResNet101V2

import tensorflow as tf


########
IMAGE_SIZE = [32,32]
CLASS=4
######


#mod =  tf.keras.applications.VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
#mod = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
#mod = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
#mod = DenseNet201(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
#mod = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
#mod = Xception(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

mod = ResNet101V2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in mod.layers:
    layer.trainable = False

x = Flatten()(mod.output)

prediction = Dense(CLASS, activation='elu')(x)

model = Model(inputs=mod.input, outputs=prediction)

############################ optimizer and learning rate ##################

adam = tf.keras.optimizers.Adam(lr=0.001)
model.compile(
  loss='categorical_crossentropy', ### for multi class calsification
  #loss='binary_crossentropy', ### for binary calsification
  optimizer=adam,
  metrics=['accuracy']
)

#########################################################################


model.optimizer.get_config()
print("\n\n")
model.summary()

171327488/171317808 [==============================] - 1s 0us/step



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 pool1_pad (ZeroPadding2D)      (None, 18, 18, 64)   0           ['conv1_conv[0][0]']             
                        

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


 conv3_block1_preact_relu (Acti  (None, 4, 4, 256)   0           ['conv3_block1_preact_bn[0][0]'] 
 vation)                                                                                          
                                                                                                  
 conv3_block1_1_conv (Conv2D)   (None, 4, 4, 128)    32768       ['conv3_block1_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv3_block1_1_bn (BatchNormal  (None, 4, 4, 128)   512         ['conv3_block1_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block1_1_relu (Activatio  (None, 4, 4, 128)   0           ['conv3_block1_1_bn[0][0]']      
 n)       

In [11]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                  # shear_range = 0.2,
                                  # zoom_range = 0.2,
                                   brightness_range=(0.4, 0.7),
                                   vertical_flip= True,
                                   horizontal_flip = True)

val_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/content/OCT_AB/train',
                                                 target_size = (32,32),
                                                 batch_size = 128,
                                                 class_mode = 'categorical')

val_set = val_datagen.flow_from_directory('/content/OCT_AB/val',
                                            target_size = (32,32),
                                            batch_size = 128,
                                            class_mode = 'categorical')

test_datagen = ImageDataGenerator(rescale = 1./255)

test_set = test_datagen.flow_from_directory('/content/OCT_AB/test',
                                           target_size = (32,32),
                                           batch_size = 1,
                                           class_mode = 'categorical')



model.optimizer.get_config()

Found 98360 images belonging to 4 classes.
Found 13113 images belonging to 4 classes.
Found 19675 images belonging to 4 classes.


{'name': 'Adam',
 'learning_rate': 0.001,
 'decay': 0.0,
 'beta_1': 0.9,
 'beta_2': 0.999,
 'epsilon': 1e-07,
 'amsgrad': False}

In [12]:
from PIL import Image

filepath = "/content/ReseNet_101V2.h5"

### example___filepath = "path to weight directory/breastCancer_geometric.h5"


checkpoint1 = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,save_weights_only=True,
                             save_best_only=True, mode='max')

log_csv = CSVLogger(r'/content/ReseNet_101V2.csv', separator=',', append=False)
### example___CSVLogger('path to logs directory/breastCancer_geometric.csv')   
    
callbacks_list = [checkpoint1,log_csv]


r = model.fit_generator(
    training_set,
    epochs=100,
    validation_data=val_set,
    steps_per_epoch = len(training_set),
    validation_steps=len(val_set),
    callbacks=callbacks_list,
    shuffle=False
    

)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/100
769/769 [==============================] - ETA: 0s - loss: 2.4686 - accuracy: 0.3719
Epoch 1: val_accuracy improved from -inf to 0.42835, saving model to /content/ReseNet_101V2.h5
769/769 [==============================] - 189s 224ms/step - loss: 2.4686 - accuracy: 0.3719 - val_loss: 1.5418 - val_accuracy: 0.4284
Epoch 2/100
769/769 [==============================] - ETA: 0s - loss: 1.4438 - accuracy: 0.4571
Epoch 2: val_accuracy improved from 0.42835 to 0.46176, saving model to /content/ReseNet_101V2.h5
769/769 [==============================] - 175s 227ms/step - loss: 1.4438 - accuracy: 0.4571 - val_loss: 1.7190 - val_accuracy: 0.4618
Epoch 3/100
769/769 [==============================] - ETA: 0s - loss: 1.5403 - accuracy: 0.4897
Epoch 3: val_accuracy improved from 0.46176 to 0.46786, saving model to /content/ReseNet_101V2.h5
769/769 [==============================] - 157s 204ms/step - loss: 1.5403 - accuracy: 0.4897 - val_loss: 1.7936 - val_accuracy: 0.4679
Epoch 4/100
7

In [13]:
model.load_weights('/content/ReseNet_101V2.h5')  ### load weights from h5 file

preds = model.evaluate_generator(test_set)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


Loss = 1.2587863206863403
Test Accuracy = 0.5374841094017029
